# Payoff Diagonal spread

Strikes differents, maturites differentes. Payoff jambe longue (T long).


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_diagonal_spread
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
K_near = spot_ref*0.98
K_far = spot_ref*1.02
T_near = 0.25
T_far = 0.75


## Payoff interactif
S0, K near/far, T near/far, sigma, r.


In [ ]:

sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='S0')
strike_near = widgets.FloatSlider(value=K_near, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='K near')
strike_far = widgets.FloatSlider(value=K_far, min=spot_ref*0.6, max=spot_ref*1.6, step=1.0, description='K far')
t_near = widgets.FloatSlider(value=T_near, min=0.05, max=1.0, step=0.05, description='T near')
t_far = widgets.FloatSlider(value=T_far, min=0.1, max=2.0, step=0.05, description='T far')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s0 = spot_slider.value
        kn, kf = strike_near.value, strike_far.value
        tn, tf = t_near.value, max(t_far.value, t_near.value + 0.01)

        fig_ts, ax_ts = plt.subplots(figsize=(8, 3))
        ax_ts.plot(close_spy.index, close_spy.values, label='SPY close (1y)')
        ax_ts.axhline(kn, color='gray', linestyle='--', label=f'K near = {kn:.2f}')
        ax_ts.axhline(kf, color='steelblue', linestyle=':', label=f'K far = {kf:.2f}')
        ax_ts.set_ylabel('Prix')
        ax_ts.set_title('Clotures SPY (strikes en pointillé)')
        ax_ts.legend(loc='best')
        fig_ts.autofmt_xdate()
        plt.show()
        plt.close(fig_ts)

        view_dyn = view_diagonal_spread(s0, kn, kf, tn, tf, r=r_slider.value, q=0.0, sigma=sigma_slider.value, option_type='call')
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(s0, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff (long far)')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(kf, color='gray', linestyle='--', label=f'K far = {kf:.2f}')
        ax.axvline(kn, color='silver', linestyle=':', label=f'K near = {kn:.2f}')
        ax.axvline(s0, color='crimson', linestyle='-.', label=f'S0 = {s0:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Diagonal spread (far leg)')
        plt.show()
        plt.close(fig)
        msg = f"""Prime ~ {premium:.4f}

- Payoff @ S0 = {payoff_s0:.4f}
- P&L net = {pnl_s0:.4f}"""

- Payoff @ S0 = {:.4f}
- P&L net = {:.4f}".format(premium, payoff_s0, pnl_s0)
        display(Markdown(msg))

for sl in (spot_slider, strike_near, strike_far, t_near, t_far, sigma_slider, r_slider, T_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_slider, strike_near, strike_far, t_near, t_far, sigma_slider, r_slider, T_slider, output]))
